In [1]:
import pandas as pd
import numpy as np
import os
import sys

from pathlib import Path
path = Path("../").resolve()  
sys.path.append(str(path))

import utils.metric as metric

In [ ]:
dataset = 'tablebench'
split = 10
df = pd.read_csv(f'../predict/{dataset}_classifier_test{split}.csv')

In [12]:
df = df.fillna('none')
df['acc_tableqa'] = df.apply(lambda x: 0 if x['ans_tableqa'] == 'none' else x['acc_tableqa'], axis=1)
df['acc_text_to_sql'] = df.apply(lambda x: 0 if x['ans_text_to_sql'] == 'none' else x['acc_text_to_sql'], axis=1)

In [ ]:
df['oracle'] = df.apply(lambda x: 1 if (x['acc_text_to_sql']==1 or x['acc_tableqa']==1) else 0, axis=1)

In [8]:
df['pred'] = df['pred'].apply(lambda x: 'none' if x == 'no' else x)

In [14]:
print('TableQA Accuracy:', df['acc_tableqa'].mean())
print('Text-to-SQL Accuracy:', df['acc_text_to_sql'].mean())
print('GPT Accuracy:', df['gpt_score'].mean())
print('Oracle Accuracy:', df['oracle'].mean())

TableQA Accuracy: 0.5613275613275613
Text-to-SQL Accuracy: 0.3477633477633478
GPT Accuracy: 0.5310245310245311
Oracle Accuracy: 0.6233766233766234


In [15]:
df.to_csv(f'../predict/{dataset}_classifier_test{split}.csv', index=False)

In [10]:
print('Text2SQL error:', (df['ans_text_to_sql'] == 'none').mean())

Text2SQL error: 0.37806637806637805
